<img src='https://gitlab.eumetsat.int/eumetlab/oceans/ocean-training/tools/frameworks/-/raw/main/img/OSI-SAF_banner.png' align='right' width='100%'/>

<a href="../Index.ipynb" target="_blank"><< Index</a>
<br>
<a href="./1_1b_OSI_SAF_wind_data_access_eumdac.ipynb">OSI SAF advanced data access via the Data Store using eumdac >></a>

<font color="#138D75">**EUMETSAT OSI SAF Training Service**</font> <br>
**Copyright:** 2024 EUMETSAT <br>
**License:** MIT

<html>
  <div style="width:100%">
    <div style="float:left"><a href="https://mybinder.org/v2/git/https%3A%2F%2Fgitlab.eumetsat.int%2Feumetlab%2Foceans%2Focean-training%2Fsensors%2Flearn-osi-saf-wind/HEAD?urlpath=%2Ftree%2F1_OSI_SAF_wind_introductory%2F1_1_OSI_SAF_wind_FTP_data_access.ipynb"><img src="https://mybinder.org/badge_logo.svg" alt="Open in Binder"></a></div>
    <div style="float:left"><p>&emsp;</p></div>
  </div>
</html>

<div class="alert alert-block alert-success">
<h3>Learn OSI SAF wind: Introductory</h3></div>

<div class="alert alert-block alert-warning">
    
<b>PREREQUISITES </b>
    
This notebook has the following prerequisites:
- **<a href="https://osi-saf.eumetsat.int/register" target="_blank">A EUMETSAT OSI SAF account</a>** if you are using or plan to use the EUMETSAT OSI SAF data.

There are no prerequisite notebooks for this module.
</div>
<hr>

# 1.1 Accessing OSI SAF wind products from KNMI FTP server

### Data used

| Product Description | Product Navigator | OSI SAF website link | OSI SAF Identifier |
|:--------------------:|:-----------------------:|:-------------:|:-----------------:|
| Metop-B ASCAT 25 km Winds | EO:EUM:DAT:METOP:OAS025 | <a href="https://osi-saf.eumetsat.int/products/osi-102-b" target="_blank">link</a> | OSI-102-b | 
| Metop-C ASCAT 25 km Winds | EO:EUM:DAT:METOP:OAS025 | <a href="https://osi-saf.eumetsat.int/products/osi-102-c" target="_blank">link</a> | OSI-102-c | 
| Metop-B ASCAT coastal Winds | EO:EUM:DAT:METOP:OSI-104 | <a href="https://osi-saf.eumetsat.int/products/osi-104-b" target="_blank">link</a> | OSI-104-b | 
| Metop-C ASCAT coastal Winds | EO:EUM:DAT:METOP:OSI-104 | <a href="https://osi-saf.eumetsat.int/products/osi-104-b" target="_blank">link</a> | OSI-104-c | 
| HY-2B 25 km wind vectors | EO:EUM:DAT:0537 | <a href="https://osi-saf.eumetsat.int/products/osi-114-a" target="_blank">link</a> | OSI-114-a | 
| HY-2B 50 km wind vectors | EO:EUM:DAT:0537 | <a href="https://osi-saf.eumetsat.int/products/osi-114-b" target="_blank">link</a> | OSI-114-b| 
| HY-2C 25 km wind vectors | EO:EUM:DAT:0539 | <a href="https://osi-saf.eumetsat.int/products/osi-115-a" target="_blank">link</a> | OSI-115-a | 
| HY-2C 50 km wind vectors | EO:EUM:DAT:0539 | <a href="https://osi-saf.eumetsat.int/products/osi-115-b" target="_blank">link</a> | OSI-115-b| 

### Learning outcomes

At the end of this notebook you will know;
* How to download Metop ASCAT winds from the KNMI FTP server

### Outline


Each product package includes:

* measurement data files (NetCDF-4 format)

<div class="alert alert-info" role="alert">
    
## <a id='TOC-TOP'></a>Contents

</div>

 0. [registering on OSI-SAF](#section0)   
 1. [Creating our workspace](#section1)
 1. [Setting query parameters](#section2)
 1. [Downloading via FTP requests](#section3)

<hr>

<div class="alert alert-info" role="alert">

## <a id='section0'></a>0. Registering on OSI-SAF
[Back to top](#TOC-TOP)

</div>

In this notebook, we will be using data from the OSI-SAF products. Before to start accessing data we produce, **<a href="https://osi-saf.eumetsat.int/register" target="_blank">sign up on our website</a>**.

<div class="alert alert-info" role="alert">

## <a id='section1'></a>1. Creating the workspace
[Back to top](#TOC-TOP)

</div>

We begin by importing all of the libraries that we need to run this notebook. If you have built your python using the environment file provided in this repository, then you should have everything you need. For more information on building environment, please see the repository **<a href="../README.md" target="_blank">README</a>**.

In [12]:
# library imports
import os           # a library that allows to access to basic operating system commands like making directories
import json         # a library that helps with JSON format files
import ftplib       # a library that allows to handle ftp requests
import gzip         # a library that allows to handle zip files
import shutil       # a library that allows to access to basic operating system commands like making directories

Next we will create a download directory to store the products we will download in this notebook.

In [13]:
download_dir = os.path.join(os.getcwd(), "products")
os.makedirs(download_dir, exist_ok=True)

<div class="alert alert-info" role="alert">

## <a id='section2'></a>2. Setting login information
[Back to top](#TOC-TOP)

</div>

We will access OSI SAF from the OSI SAF Wind processing centre FTP server.

In order to allow us to download data, we need to provide our credentials. We can do this in two ways; either by creating a file called `.eumetsat_osi_saf_wind_credentials` in our home directory (*option 1 - recommended*) or by supplying our credentials directly in this script (*option 2*). 

#### Option 1: creating  `.eumetsat_osi_saf_wind_credentials` in our home directory

For most computer systems the home directory can be found at the path \user\username, /users/username, or /home/username depending on your operating system.

In this file we need to add the following information exactly as follows;

```
{
"username": "<provided_username>",
"password": "<provided_password>"
}
```

You must replace `<provided_username>` and `<provided_password>` with the information provided to you after you have registered to the OSI SAF website and mentionned your interest in Sea Ice data. 

Once you have created an <a href="https://osi-saf.eumetsat.int/register" target="_blank">EUMETSAT OSI SAF account</a>, the generic password and credentials to access OSI SAF Wind server should have been sent to you via email.

Make sure to save the file without any kind of extension.

Once you have done this, you can read in your credentials using the commands in the following cell. These will be used to generate a time-limited token, which will refresh itself when it expires.

Reading credential information

In [14]:
# read credentials
with open(os.path.join(os.path.expanduser("~"),'.eumetsat_osi_saf_wind_credentials')) as json_file:
    credentials = json.load(json_file)

Loading credentials

In [15]:
username = credentials['username'] 
password = credentials['password']

#### Option 2: provide credentials directly

You can provide your credentials directly as follows; by uncommenting the following lines. 

In [16]:
#username = "<provided_username>"
#password = "<provided_password>"

You must replace <provided_username> and <provided_password> with the information provided to you after you have registered to the OSI SAF website and mentionned your interest in Sea Ice data.

Once you have created an EUMETSAT OSI SAF account, the generic password and credentials to access OSI SAF Wind server should have been sent to you via email.

Note: this method is convenient in the short term, but is not really recommended as you have to put your provided username and password in this notebook, and run the risk of accidentally sharing them. This method also requires you to authenticate on a notebook-by-notebook basis.

<div class="alert alert-info" role="alert">

## <a id='section3'></a>3. FTP access winds
[Back to top](#TOC-TOP)

</div>

#### ASCAT B wind 25 km data
Let's start by creating a specific folder to hold the Metop-B/Ascat Wind products

In [17]:
download_dir = os.path.join(os.getcwd(), "products" ,"ascat_b", "25")
os.makedirs(download_dir, exist_ok=True)

Set the download path: <br> 
To do so, you need the name of the orbit file you want to download (remember, with ASCAT 1 orbit = two swaths). To do so, the most efficient way is to explore the KNMI FTP server with a FTP client. The Metop-B/Ascat products are to be found in the directory ftppro.knmi.nl/scat/netcdf/ascat_b/
As shown in the example below, you can just use the orbit identifier to get it.<br>
__Note that the FTP server only gives access to the last three days of wind data.__ 

In [18]:
path = '/scat/netcdf/ascat_b/'
file_pattern = 'eps_o_250_3301_ovw.l2.nc.gz'

In [19]:
# connect ftp and download file
ftp = ftplib.FTP("ftppro.knmi.nl") 
ftp.login(username, password)
ftp.cwd(path)
files = ftp.nlst()
for file in files:
    if file_pattern in file and 'md5' not in file:
        filename = file
ftp.retrbinary("RETR " + filename, open(os.path.join(os.getcwd(), 'products', 'ascat_b', '25', filename), 'wb').write)
ftp.quit()

'221 Goodbye.'

Unzip the data file

In [20]:
with open(os.path.join(os.getcwd(), 'products', 'ascat_b', '25', filename[:-3]), 'wb') as f_out:
    with gzip.open(os.path.join(os.getcwd(), 'products', 'ascat_b', '25', filename), 'rb') as f_in:
        shutil.copyfileobj(f_in, f_out)

#### ASCAT C wind 25 km data
Let's create a specific folder to hold the Metop-C/Ascat Wind products

In [21]:
download_dir = os.path.join(os.getcwd(), "products", "ascat_c", "25")
os.makedirs(download_dir, exist_ok=True)

Set the download path: <br>
Proceed as for ASCAT B 25 km data to get the file name you want to download. __Keep in mind the server store 3 days of data__.

In [22]:
path = '/scat/netcdf/ascat_c/'
file_pattern = 'eps_o_250_3301_ovw.l2.nc.gz'

In [23]:
# connect ftp and download file
ftp = ftplib.FTP("ftppro.knmi.nl") 
ftp.login(username, password)
ftp.cwd(path)
files = ftp.nlst()
for file in files:
    if file_pattern in file and 'md5' not in file:
        filename = file
ftp.retrbinary("RETR " + filename, open(os.path.join(os.getcwd(), 'products', 'ascat_c', '25', filename), 'wb').write)
ftp.quit()

'221 Goodbye.'

Unzip the data file

In [24]:
with open(os.path.join(os.getcwd(), 'products', 'ascat_c', '25', filename[:-3]), 'wb') as f_out:
    with gzip.open(os.path.join(os.getcwd(), 'products', 'ascat_c', '25', filename), 'rb') as f_in:
        shutil.copyfileobj(f_in, f_out)

#### ASCAT B coastal wind data

The coastal areas are tricky for satellite measurements. Usually they are not taken into account on the data distribution for prediction has the coast generates artefacts. However, observing the wind on coastal areas is important to some users, that's why the OSI SAF wind team process and distributes these "coastal wind" dataset out from the same ASCAT instruments. Their higher resolution makes it more coslty to use, but allow getting reliable data from closer to the coasts.
<br><br>
Let's start by creating a specific folder to hold the Metop-B/Ascat costal Wind products

In [25]:
download_dir = os.path.join(os.getcwd(), "products", "ascat_b", "coa")
os.makedirs(download_dir, exist_ok=True)

Proceed as for ASCAT B 25km and ASCAT C 25kmv to get your file_pattern correct. (cf. upper)

In [26]:
path = '/scat/netcdf/ascat_b/'
file_pattern = 'eps_o_coa_3301_ovw.l2.nc.gz'

In [27]:
# connect ftp and download file
ftp = ftplib.FTP("ftppro.knmi.nl") 
ftp.login(username, password)
ftp.cwd(path)
files = ftp.nlst()
for file in files:
    if file_pattern in file and 'md5' not in file:
        filename = file
ftp.retrbinary("RETR " + filename, open(os.path.join(os.getcwd(), 'products', 'ascat_b', 'coa', filename), 'wb').write)
ftp.quit()

'221 Goodbye.'

Unzip the data file

In [28]:
with open(os.path.join(os.getcwd(),'products', 'ascat_b', 'coa', filename[:-3]), 'wb') as f_out:
    with gzip.open(os.path.join(os.getcwd(), 'products', 'ascat_b', 'coa', filename), 'rb') as f_in:
        shutil.copyfileobj(f_in, f_out)

#### ASCAT C coastal wind data
Let's start by creating a specific folder to hold the Metop-C/Ascat costal Wind products

In [29]:
download_dir = os.path.join(os.getcwd(), "products", "ascat_c", "coa")
os.makedirs(download_dir, exist_ok=True)

Proceed as for ASCAT B 25km and ASCAT C 25kmv to get your file_pattern correct. (cf. upper)

In [30]:
path = '/scat/netcdf/ascat_c/'
file_pattern = 'eps_o_coa_3301_ovw.l2.nc.gz'

In [31]:
# connect ftp and download file
ftp = ftplib.FTP("ftppro.knmi.nl") 
ftp.login(username, password)
ftp.cwd(path)
files = ftp.nlst()
for file in files:
    if file_pattern in file and 'md5' not in file:
        filename = file
ftp.retrbinary("RETR " + filename, open(os.path.join(os.getcwd(), 'products','ascat_c', 'coa', filename), 'wb').write)
ftp.quit()

'221 Goodbye.'

Unzip the data file

In [32]:
with open(os.path.join(os.getcwd(), 'products', 'ascat_c', 'coa', filename[:-3]), 'wb') as f_out:
    with gzip.open(os.path.join(os.getcwd(), 'products', 'ascat_c', 'coa', filename), 'rb') as f_in:
        shutil.copyfileobj(f_in, f_out)

#### HY-2B 25 km wind data
Additionally to winds observed from METOP series satellites, OSI SAF process and distributes wind observation from the Chinese National Satellite Ocean Application Service (NSOAS)'s Hai Yang (HY) series of satellites. The wind observation instrument onboard these satellite is a scatterometer named HSCAT. <br><br>

Let's start by creating a specific folder to hold the HY 2B 25 km Wind products

In [33]:
download_dir = os.path.join(os.getcwd(), "products", "hy_2b", "25")
os.makedirs(download_dir, exist_ok=True)

In [34]:
path = '/scat/netcdf/hy2b'
file_pattern = 'o_250_2204_ovw_l2.nc.gz'

In [35]:
# connect ftp and download file
ftp = ftplib.FTP("ftppro.knmi.nl") 
ftp.login(username, password)
ftp.cwd(path)
files = ftp.nlst()
for file in files:
    if file_pattern in file and 'md5' not in file:
        filename = file
ftp.retrbinary("RETR " + filename, open(os.path.join(os.getcwd(), 'products', 'hy_2b', '25', filename), 'wb').write)
ftp.quit()

'221 Goodbye.'

In [36]:
with open(os.path.join(os.getcwd(), 'products', 'hy_2b', '25', filename[:-3]), 'wb') as f_out:
    with gzip.open(os.path.join(os.getcwd(), 'products', 'hy_2b', '25', filename), 'rb') as f_in:
        shutil.copyfileobj(f_in, f_out)

#### HY-2B 50 km wind data
Let's start by creating a specific folder to hold the HY 2B 50 km Wind products

In [37]:
download_dir = os.path.join(os.getcwd(), "products", "hy_2b", "50")
os.makedirs(download_dir, exist_ok=True)

In [38]:
path = '/scat/netcdf/hy2b'
file_pattern = 'o_500_2204_ovw_l2.nc.gz'

In [39]:
# connect ftp and download file
ftp = ftplib.FTP("ftppro.knmi.nl") 
ftp.login(username, password)
ftp.cwd(path)
files = ftp.nlst()
for file in files:
    if file_pattern in file and 'md5' not in file:
        filename = file
ftp.retrbinary("RETR " + filename, open(os.path.join(os.getcwd(), 'products', 'hy_2b', '50', filename), 'wb').write)
ftp.quit()

'221 Goodbye.'

In [40]:
with open(os.path.join(os.getcwd(), 'products', 'hy_2b', '50', filename[:-3]), 'wb') as f_out:
    with gzip.open(os.path.join(os.getcwd(), 'products', 'hy_2b', '50', filename), 'rb') as f_in:
        shutil.copyfileobj(f_in, f_out)

#### HY-2C 25 km wind data
Similarly to the METOP series, HY one have several satellite orbiting simultineously. Currently HY-2B, HY-2C and HY-2D are in service. OSI SAF process and distributes data out from these three satellites observations.
<br><br>
Let's start by creating a specific folder to hold the HY 2C 25 km Wind products

In [41]:
download_dir = os.path.join(os.getcwd(), "products", "hy_2c", "25")
os.makedirs(download_dir, exist_ok=True)

In [42]:
path = '/scat/netcdf/hy2c'
file_pattern = 'o_250_2204_ovw_l2.nc.gz'

In [43]:
# connect ftp and download file
ftp = ftplib.FTP("ftppro.knmi.nl") 
ftp.login(username, password)
ftp.cwd(path)
files = ftp.nlst()
for file in files:
    if file_pattern in file and 'md5' not in file:
        filename = file
ftp.retrbinary("RETR " + filename, open(os.path.join(os.getcwd(),'products', 'hy_2c', '25', filename), 'wb').write)
ftp.quit()

'221 Goodbye.'

In [44]:
with open(os.path.join(os.getcwd(), 'products', 'hy_2c', '25', filename[:-3]), 'wb') as f_out:
    with gzip.open(os.path.join(os.getcwd(), 'products', 'hy_2c', '25', filename), 'rb') as f_in:
        shutil.copyfileobj(f_in, f_out)

#### HY-2C 50 km wind data
Let's start by creating a specific folder to hold the HY 2C 50 km Wind products

In [45]:
download_dir = os.path.join(os.getcwd(), "products", "hy_2c", "50")
os.makedirs(download_dir, exist_ok=True)

In [46]:
path = '/scat/netcdf/hy2c'
file_pattern = 'o_500_2204_ovw_l2.nc.gz'

In [47]:
# connect ftp and download file
ftp = ftplib.FTP("ftppro.knmi.nl") 
ftp.login(username, password)
ftp.cwd(path)
files = ftp.nlst()
for file in files:
    if file_pattern in file and 'md5' not in file:
        filename = file
ftp.retrbinary("RETR " + filename, open(os.path.join(os.getcwd(), 'products', 'hy_2c', '50', filename), 'wb').write)
ftp.quit()

'221 Goodbye.'

In [48]:
with open(os.path.join(os.getcwd(), 'products', 'hy_2c', '50', filename[:-3]), 'wb') as f_out:
    with gzip.open(os.path.join(os.getcwd(), 'products', 'hy_2c', '50', filename), 'rb') as f_in:
        shutil.copyfileobj(f_in, f_out)

<hr>
<a href="../Index.ipynb" target="_blank"><< Index</a>
<br>
<a href="./1_1b_OSI_SAF_wind_data_access_eumdac.ipynb">OSI SAF advanced data access via the Data Store using eumdac >></a>
<hr>
<a href="https://gitlab.eumetsat.int/eumetlab/ocean">View on GitLab</a> | <a href="https://training.eumetsat.int/">EUMETSAT Training</a> | <a href=mailto:ops@eumetsat.int>Contact helpdesk for support </a> | <a href=mailto:training@eumetsat.int>Contact our training team to collaborate on and reuse this material</a></span></p>